<a href="https://colab.research.google.com/github/sju0924/sju0924/blob/master/smart_AIoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [68]:
# Define the NLP model as a PyTorch neural network
class LEDControlModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, embedding_dim):
        super(LEDControlModel, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        #embedding = self.embedding(input).view(1,-1)
        #print("embedding: ", embedding.size())
        hidden = F.relu(self.fc1(input))
        output = self.fc2(hidden)
        return output

In [69]:
# Initialize the NLP model
input_size = 20  # number of words in vocabulary
embedding_dim = 5  # size of word embeddings
hidden_size = 500
output_size = 32  # number of possible LED states
model = LEDControlModel(input_size, hidden_size, output_size, embedding_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [66]:
# Train the NLP model on the input data
input_data = [["turn", "on", "led", "number", "7"], ["turn", "off", "led", "number", "12"], 
              ["set", "led", "number", "3", "to", "high"], ["set", "led", "number", "16", "to", "low"],
              ['turn', 'on',  'led',  '3', 'and' ,'LED', '4'],['turn', 'off', 'LED', '5', 'to', 'LED', '8'],
              ['turn', 'on', 'all', 'leds'], ['turn', 'off', 'all', 'leds'],
              ["set", "led", "number", "11", "to", "13", "on"], ["set", "led", "number", "3", "to","6", "off"]]
target_data = [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
               [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
               [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]               
               ]

freq = {}
for sentence in input_data:
  for word in sentence:
    if word in freq:
      freq[word] += 1
    else:
      freq[word] = 1

word_to_index = {}
for i in range (17):
  word_to_index[str(i)] = i

i = 17
for word in freq:
  if word.isdigit():
    continue
  word_to_index[word] = i
  i += 1

print(word_to_index)


{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, 'turn': 17, 'on': 18, 'led': 19, 'number': 20, 'off': 21, 'set': 22, 'to': 23, 'high': 24, 'low': 25, 'and': 26, 'LED': 27, 'all': 28, 'leds': 29}


In [86]:
for epoch in range(1000):
    total_loss = 0
    for example, target in zip(input_data, target_data):
        optimizer.zero_grad()
        input = torch.tensor([word_to_index[word] for word in example]+[0] *(input_size - len(example)), dtype=torch.float32)
        output = model(input)
        loss = criterion(output, torch.tensor(target, dtype=torch.float32))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print('Epoch [{}/100], Loss: {:.4f}'.format(epoch + 1, total_loss / len(input_data)))

Epoch [10/100], Loss: 16.5673
Epoch [20/100], Loss: 16.5670
Epoch [30/100], Loss: 16.5668
Epoch [40/100], Loss: 16.5666
Epoch [50/100], Loss: 16.5665
Epoch [60/100], Loss: 16.5664
Epoch [70/100], Loss: 16.5664
Epoch [80/100], Loss: 16.5663
Epoch [90/100], Loss: 16.5663
Epoch [100/100], Loss: 16.5663
Epoch [110/100], Loss: 16.5663
Epoch [120/100], Loss: 16.5663
Epoch [130/100], Loss: 16.5663
Epoch [140/100], Loss: 16.5663
Epoch [150/100], Loss: 16.5663
Epoch [160/100], Loss: 16.5663
Epoch [170/100], Loss: 16.5663
Epoch [180/100], Loss: 16.5663
Epoch [190/100], Loss: 16.5663
Epoch [200/100], Loss: 16.5663
Epoch [210/100], Loss: 16.5663
Epoch [220/100], Loss: 16.5663
Epoch [230/100], Loss: 16.5663
Epoch [240/100], Loss: 16.5663
Epoch [250/100], Loss: 16.5663
Epoch [260/100], Loss: 16.5663
Epoch [270/100], Loss: 16.5663
Epoch [280/100], Loss: 16.5663
Epoch [290/100], Loss: 16.5663
Epoch [300/100], Loss: 16.5663
Epoch [310/100], Loss: 16.5663
Epoch [320/100], Loss: 16.5663
Epoch [330/100], 